In [95]:
import pandas as pd 
import yfinance as yf
import numpy as np
import datetime


In [96]:
prices = yf.download(['GENTERA.MX'], start='2021-01-01', end='2025-01-01')['Close']
prices.head()

[*********************100%***********************]  1 of 1 completed


Ticker,GENTERA.MX
Date,
2021-01-04,9.214066
2021-01-05,8.959209
2021-01-06,9.018022
2021-01-07,9.478725
2021-01-08,9.321890


In [ ]:
def get_historical_price(ticker: str, start: str):
    end = start+datetime.timedelta(days=4)
    price=pd.DataFrame(yf.Ticker(ticker).history(start=start, end=end)["Close"])
    return price.iloc[0, 0]

In [98]:
def get_financial (ticker: str):
    info = yf.Ticker("GENTERA.MX")
    balance_g = info.balance_sheet.iloc[:,0:-1]
    estados_r = info.financials.iloc[:,0:-1]
    
    return balance_g, estados_r


In [99]:
balance_g, estados_r = get_financial("GENTERA.MX")


In [100]:
net_income = estados_r.loc['Net Income Common Stockholders'] 
net_income

2024-12-31    6004665558.0
2023-12-31    4722486795.0
2022-12-31    4554000000.0
2021-12-31    2347000000.0
Name: Net Income Common Stockholders, dtype: object

In [101]:
Shares = balance_g.loc['Ordinary Shares Number']
Shares

2024-12-31    1579239112.0
2023-12-31    1584093876.0
2022-12-31    1582743876.0
2021-12-31    1587593876.0
Name: Ordinary Shares Number, dtype: object

In [102]:
EPS = net_income/ Shares
EPS

2024-12-31    3.802252
2023-12-31    2.981191
2022-12-31    2.877282
2021-12-31    1.478338
dtype: object

In [103]:
EPS.mean()

2.78476575884277

A cada accionista le tocaria en promedio 2.78 dolares

---

In [104]:
prices = [get_historical_price("GENTERA.MX", date) for date in EPS.index]
prices

[24.010000228881836, 23.010000228881836, 22.0, 12.919297218322754]

In [105]:
PER = prices / EPS
PER

2024-12-31    6.314678
2023-12-31    7.718391
2022-12-31    7.646106
2021-12-31     8.73907
dtype: object

Se puede creer que esta acción está infravalorada

---

In [106]:
Liabilities= balance_g.loc['Total Liabilities Net Minority Interest']
Assets = balance_g.loc['Total Assets']

In [107]:
Solvency =  Assets / Liabilities
Solvency

2024-12-31     1.45738
2023-12-31    1.474793
2022-12-31    1.515999
2021-12-31    1.546547
dtype: object

In [112]:
Solvency.mean()

1.498679462269381

En cuanto a solvencia , esta acción está normal.

---

In [108]:
Equity = balance_g.loc['Stockholders Equity']

In [111]:
ROE = net_income / Equity
ROE*100

2024-12-31    19.306783
2023-12-31    18.350444
2022-12-31    19.309701
2021-12-31    11.053028
dtype: object

---